# Ex2 - Getting and Knowing your Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkFiles
spark = SparkSession.builder.getOrCreate()

23/05/26 21:46:14 WARN Utils: Your hostname, taliandorogd resolves to a loopback address: 127.0.1.1; using 192.168.0.32 instead (on interface wlp5s0)
23/05/26 21:46:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/26 21:46:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

In [2]:
spark.sparkContext.addFile("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv")

### Step 3. Assign it to a variable called chipo.

In [3]:
chipo = spark.read.csv(SparkFiles.get("chipotle.tsv"), sep = '\t', header = True)

### Step 4. See the first 10 entries

In [4]:
chipo.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

### Step 5. What is the number of observations in the dataset?

In [5]:
# Solution 1
chipo.count()


4622

In [6]:
# Solution 2
chipo.describe().show()


+-------+-----------------+------------------+-----------------+--------------------+----------+
|summary|         order_id|          quantity|        item_name|  choice_description|item_price|
+-------+-----------------+------------------+-----------------+--------------------+----------+
|  count|             4622|              4622|             4622|                4622|      4622|
|   mean|927.2548680225011|1.0757247944612722|             null|                null|      null|
| stddev|528.8907955866096|0.4101863342575333|             null|                null|      null|
|    min|                1|                 1|6 Pack Soft Drink|                NULL|    $1.09 |
|    max|              999|                 8|Veggie Soft Tacos|[[Tomatillo-Red C...|    $9.39 |
+-------+-----------------+------------------+-----------------+--------------------+----------+



### Step 6. What is the number of columns in the dataset?

In [7]:
len(chipo.columns)

5

### Step 7. Print the name of all the columns.

In [8]:
for i, col in enumerate(chipo.columns):
    print(f"Columns {i + 1}: {col}")

Columns 1: order_id
Columns 2: quantity
Columns 3: item_name
Columns 4: choice_description
Columns 5: item_price


### Step 8. How is the dataset indexed?

In [9]:
# Not applicable

### Step 9. Which was the most-ordered item? 

In [10]:
from pyspark.sql.types import IntegerType
chipo = chipo\
    .withColumn("quantity", chipo["quantity"].cast(IntegerType()))

chipo\
    .groupby("item_name")\
    .sum("quantity")\
    .withColumnRenamed("sum(quantity)", "quantity")\
    .orderBy("quantity", ascending = False)\
    .select("item_name")\
    .show(1)

+------------+
|   item_name|
+------------+
|Chicken Bowl|
+------------+
only showing top 1 row



### Step 10. For the most-ordered item, how many items were ordered?

In [11]:
chipo\
    .groupby("item_name")\
    .sum("quantity")\
    .withColumnRenamed("sum(quantity)", "quantity")\
    .orderBy("quantity", ascending = False)\
    .select("quantity")\
    .show(1)

+--------+
|quantity|
+--------+
|     761|
+--------+
only showing top 1 row



### Step 11. What was the most ordered item in the choice_description column?

In [12]:
chipo\
    .groupby("choice_description")\
    .sum("quantity")\
    .withColumnRenamed("sum(quantity)", "quantity")\
    .orderBy("quantity", ascending = False)\
    .filter(chipo.choice_description != "NULL")\
    .show(1)
    # .select("choice_description")\

+------------------+--------+
|choice_description|quantity|
+------------------+--------+
|       [Diet Coke]|     159|
+------------------+--------+
only showing top 1 row



### Step 12. How many items were ordered in total?

In [13]:
chipo\
    .groupby()\
    .sum("quantity")\
    .show()

+-------------+
|sum(quantity)|
+-------------+
|         4972|
+-------------+



### Step 13. Turn the item price into a float

#### Step 13.a. Check the item price type

In [14]:
[col[1] for col in chipo.dtypes if col[0] == "item_price"]

['string']

In [15]:
chipo.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

#### Step 13.b. Create a lambda function and change the type of item price

In [16]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType

chipo = chipo\
    .withColumn("item_price", udf(lambda s: s[1:])(col("item_price")).cast(FloatType()))

#### Step 13.c. Check the item price type

In [17]:
chipo.dtypes

[('order_id', 'string'),
 ('quantity', 'int'),
 ('item_name', 'string'),
 ('choice_description', 'string'),
 ('item_price', 'float')]

### Step 14. How much was the revenue for the period in the dataset?

In [21]:
chipo\
.withColumn("product", col("quantity") * col("item_price"))\
.groupby()\
.sum("product")\
.show()

+----------------+
|    sum(product)|
+----------------+
|39237.0197327137|
+----------------+



### Step 15. How many orders were made in the period?

In [24]:
chipo.select("order_id").distinct().count()

1834

### Step 16. What is the average revenue amount per order?

In [33]:
chipo\
.withColumn("product", col("quantity") * col("item_price"))\
.groupby("order_id")\
.sum("product")\
.withColumnRenamed("sum(product)", "product")\
.groupby()\
.avg("product")\
.show()

+------------------+
|      avg(product)|
+------------------+
|21.394231042919138|
+------------------+



### Step 17. How many different items are sold?

In [26]:
chipo.select("item_name").distinct().count()

50